# Time Series Anomaly Detecion: Detection of Anomalous Drops with Limited Features and Sparse Examples in Noisy Highly Periodic Data

Paper is [here](https://static.googleusercontent.com/media/research.google.com/ru//pubs/archive/dfd834facc9460163438b94d53b36f51bb5ea952.pdf), additional information is [here](https://nau-datascience.github.io/Time-Series-Anomaly-Detection).  
Anomaly detection in network traffic. Only two attributes - time stamp and bytes per second sampled at 5 minute interval. In total - 14 different data streams. Each data stream gets its own model.  
Three models are considered - DNN, RNN and LSTM. Not clear how many input features they used, I will use 50 because this is what is used in examples on the page with additional info.  
**Models**  
1. DNN model - DNN Regressor - fully-connected model. In total, 10 layers of 200 neurons each, batch size 200, trained for 1200 steps, single linear output layer
2. RNN model - 10 layer RNN with 75 units in hidden size, batch size 200 at 2500 steps.
3. LSTM - 10 layer model with 70 neurons, single lienar output layer.

Authors report that RNN/LSTM did not perform much better compared to DNN. Summary*:
1. DNN regressor: 3.702000e-04 gFLOPs (forward)
2. RNN mode: 5.347575e-03 gFLOPs (forward)
3. LSTM model: 1.863407e-02 gFLOPs (forward)


*FLOPs - Approximate number of multiply-add operations to perform a task.

In [21]:
import contextlib
import os, sys
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), 'python'))
from nns.nns import ModelSummary
from nns.model import Model
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, LSTM, Activation, SimpleRNN

### DNN Regressor

In [8]:
class DNNModel(Model):
    def __init__(self):
        super().__init__('DNNModel')

    def create(self):
        model = models.Sequential([Model.Input((50,))], name=self.name)
        for i in range(10):
                model.add(Model.Dense(200, name='dense{}'.format(i+1)))
        model.add(Model.Dense(1, activation=None, name='output'))
        return model

In [9]:
ModelSummary(DNNModel().create()).summary()

DNNModel
       name        gflops  nparams  params_mb
0    dense1  1.000000e-05    10200   0.038910
1    dense2  4.000000e-05    40200   0.153351
2    dense3  4.000000e-05    40200   0.153351
3    dense4  4.000000e-05    40200   0.153351
4    dense5  4.000000e-05    40200   0.153351
5    dense6  4.000000e-05    40200   0.153351
6    dense7  4.000000e-05    40200   0.153351
7    dense8  4.000000e-05    40200   0.153351
8    dense9  4.000000e-05    40200   0.153351
9   dense10  4.000000e-05    40200   0.153351
10   output  2.000000e-07      201   0.000767
11    TOTAL  3.702000e-04   372201   1.419834


### RNN model

In [14]:
class RNNModel(Model):
    def __init__(self):
        super().__init__('RNNModel')

    def create(self):
        model = models.Sequential([Model.Input((50,1))], name=self.name)
        for i in range(9):
                model.add(SimpleRNN(75, return_sequences=True, name='rnn0{}'.format(i+1)))
        model.add(SimpleRNN(75, return_sequences=False, name='rnn10'))
        model.add(Model.Dense(1, activation=None, name='output'))
        return model

In [15]:
ModelSummary(RNNModel().create()).summary()

RNNModel
                 name        gflops  nparams  params_mb
0   rnn01 (SimpleRNN)  2.850000e-04     5775   0.022030
1   rnn02 (SimpleRNN)  5.625000e-04    11325   0.043201
2   rnn03 (SimpleRNN)  5.625000e-04    11325   0.043201
3   rnn04 (SimpleRNN)  5.625000e-04    11325   0.043201
4   rnn05 (SimpleRNN)  5.625000e-04    11325   0.043201
5   rnn06 (SimpleRNN)  5.625000e-04    11325   0.043201
6   rnn07 (SimpleRNN)  5.625000e-04    11325   0.043201
7   rnn08 (SimpleRNN)  5.625000e-04    11325   0.043201
8   rnn09 (SimpleRNN)  5.625000e-04    11325   0.043201
9   rnn10 (SimpleRNN)  5.625000e-04    11325   0.043201
10             output  7.500000e-08       76   0.000290
11              TOTAL  5.347575e-03   107776   0.411133


### LSTM model

In [19]:
class LSTMModel(Model):
    def __init__(self):
        super().__init__('LSTMModel')

    def create(self):
        model = models.Sequential([Model.Input((50,1))], name=self.name)
        for i in range(9):
                model.add(LSTM(70, return_sequences=True, name='lstm0{}'.format(i+1)))
        model.add(LSTM(70, return_sequences=False, name='lstm10'))
        model.add(Model.Dense(1, activation=None, name='output'))
        return model

In [22]:
with contextlib.redirect_stderr(None):
    ModelSummary(LSTMModel().create()).summary()

LSTMModel
             name        gflops  nparams  params_mb
0   lstm01 (LSTM)  9.940000e-04    20160   0.076904
1   lstm02 (LSTM)  1.960000e-03    39480   0.150604
2   lstm03 (LSTM)  1.960000e-03    39480   0.150604
3   lstm04 (LSTM)  1.960000e-03    39480   0.150604
4   lstm05 (LSTM)  1.960000e-03    39480   0.150604
5   lstm06 (LSTM)  1.960000e-03    39480   0.150604
6   lstm07 (LSTM)  1.960000e-03    39480   0.150604
7   lstm08 (LSTM)  1.960000e-03    39480   0.150604
8   lstm09 (LSTM)  1.960000e-03    39480   0.150604
9   lstm10 (LSTM)  1.960000e-03    39480   0.150604
10         output  7.000000e-08       71   0.000271
11          TOTAL  1.863407e-02   375551   1.432613
